In [1]:
import pandas as pd
import ipaddress
import numpy as np
import pandasql as ps

In [5]:
filename = '~/Downloads/log20030630/log20030630.csv'
#parsing_dates didn't work.
#couldn't use between times. 

In [6]:
def pipeline(filename):
    df = pd.read_csv(filename,nrows=50000)
    df['ip']= to_ipaddress(df['ip'])
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = [date.date() for date in df['date']]
    return df[['ip', 'date','cik','accession']]

def to_ipaddress(arr): 
    arr = [np.char.replace(ip, ip[-3:],'000') for ip in arr]
    return [int(ipaddress.IPv4Address(x)) for x in arr]

def setupiptable(filename):
    df = pd.read_csv(filename)
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: int(ipaddress.IPv4Address(x)))
    df.iloc[:,1] = df.iloc[:,1].apply(lambda x: int(ipaddress.IPv4Address(x)))
    df.columns = ["from_ip","to_ip","code","country"]
    return df.drop(columns=['code'])

In [12]:
df = pipeline(filename)

FileNotFoundError: [Errno 2] File b'/Users/albertmorrow/Downloads/log20030630/log20030630.csv' does not exist: b'/Users/albertmorrow/Downloads/log20030630/log20030630.csv'

In [8]:
dfgeo = setupiptable('~/Downloads/IP2LOCATION-LITE-DB1.CSV')


,from_ip,to_ip,country
0,16777216,16777471,United States of America


In [9]:
sqlcode = '''
    SELECT * 
    FROM df, dfgeo
    WHERE ip BETWEEN from_ip AND to_ip
'''
newdf = ps.sqldf(sqlcode, locals()).drop(columns=['from_ip','to_ip'])

In [11]:
newdf['country'].value_counts()

United States of America    31129
Puerto Rico                  4799
India                        3709
Japan                        2131
Canada                       1541
                            ...  
Malta                           1
Guatemala                       1
Jordan                          1
Ukraine                         1
Brunei Darussalam               1
Name: country, Length: 72, dtype: int64

In [ ]:
sqlcode_topcik = '''
    SELECT cik, accession, country, COUNT(*)
    FROM newdf
    GROUP BY cik
    ORDER BY 4 DESC
    LIMIT 20;
'''
#topcikdf= ps.sqldf(sqlcode_topcik, locals())

In [ ]:
sqlcode_topcikbycountry = '''
    SELECT cik, accession, country, row_number() over (partition by country order by COUNT(*) desc) as country_rank 
    from df;
'''
#topcikbycountrydf= ps.sqldf(sqlcode_topcikbycountry, locals())

In [2]:
sqlcode_topdoc = '''
    SELECT accession, country, COUNT(*)
    FROM newdf
    GROUP BY accession
    ORDER BY 3 DESC
    LIMIT 20;
'''
#topdocdf = ps.sqldf(sqlcode_topdoc, locals())

In [ ]:
sqlcode_topdocbycountry = '''
    SELECT COUNT(accession), country
    FROM newdf
    GROUP BY country
    ORDER BY 1 DESC
    LIMIT 20;
'''
#topdocbycountrydf= ps.sqldf(sqlcode_topdocbycountry, locals())

In [39]:
countrydf2003['Normalized'] = countrydf2003['Count'].value_counts(normalize=True)
countrydf2010['Normalized'] = countrydf2010['Count'].value_counts(normalize=True)
countrydf2017['Normalized'] = countrydf2017['Count'].value_counts(normalize=True)